In [1]:
import os
subpath = './input'
all_file = os.listdir(subpath)
all_file

['.ipynb_checkpoints',
 'sample_submission.csv.7z',
 'test',
 'test.json',
 'test.json.7z',
 'train',
 'train.json',
 'train.json.7z']

In [1]:
import pandas as pd
import numpy as np

In [17]:
df = pd.DataFrame({'id1':[1,2,1,2,1,2,1,2], 'id2':[0,1,1,1,2,3,1,2]})
df

,id1,id2
0,1,0
1,2,1
2,1,1
3,2,1
4,1,2
5,2,3
6,1,1
7,2,2


In [8]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 1
cutoff_hi = 3

In [15]:
df['is_iceberg_median']

0    0.5
1    1.5
2    1.0
3    1.5
4    1.5
5    2.5
6    1.0
7    2.0
Name: is_iceberg_median, dtype: float64

In [24]:
df['is_iceberg'] = np.where(np.all(df.iloc[:,:] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(df.iloc[:, :] < cutoff_hi, axis=1), 0, df['is_iceberg_median']))
df['is_iceberg']
#concat_sub[['id', 'is_iceberg']].to_csv('stack_pushout_median.csv', 
#                                        index=False, float_format='%.6f')


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    1.0
6    0.0
7    1.0
Name: is_iceberg, dtype: float64

In [18]:
np.all(df.iloc[:,:] > cutoff_lo, axis=1)

0    False
1    False
2    False
3    False
4    False
5     True
6    False
7     True
dtype: bool

In [21]:
np.all(df.iloc[:, :] < cutoff_hi, axis=1)

0     True
1     True
2     True
3     True
4     True
5    False
6     True
7     True
dtype: bool

In [22]:
df['is_iceberg_median'] = df.iloc[:, :].median(axis=1)
df['is_iceberg_median']

0    0.5
1    1.5
2    1.0
3    1.5
4    1.5
5    2.5
6    1.0
7    2.0
Name: is_iceberg_median, dtype: float64

In [23]:
np.where(np.all(df.iloc[:, :] < cutoff_hi, axis=1), 0, df['is_iceberg_median'])

array([0. , 0. , 0. , 0. , 0. , 2.5, 0. , 0. ])

In [ ]:
# np.where(조건, True 일 때 출력값, False 일 때 출력값)
# np.all(조건) - 배열의 모든 데이터가 조건과 맞으면 True 하나라도 다르면 False
# np.any(조건) - 배열의 데이터 중 조건과 맞는 데이터가 있으면 True 전혀 없으면 False
# np.isnan(arr) - 배열의 데이터 중 NaN 값인 요소는 True 아니면 False로 변환
# np.isfinite(arr) - 배열의 데이터 중 셀수 있는 데이터를 True 아니면 False로 변환


In [25]:
# Random initialization
import numpy as np
np.random.seed(98643)
import tensorflow as tf
# 할당에 대한 TF 경고를 숨기려면 이 메모를 해제하십시오.
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 이미지 종속성을 지웁니다.
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma, denoise_tv_bregman, denoise_nl_means)
# restoration : 복구
# denoise : 노이즈 제거
# denoise_tv_chambolle : n차원 이미지에 대해 전체 변형 노이즈 제거를 수행합니다.
# denoise_bilateral : 양방향 필터를 사용하여 이미지를 노이즈 제거합니다.
# denoise_wavelet : 이미지에 웨이블릿 노이즈 제거를 수행합니다.
# estimate_sigma : (가우스) 노이즈 표준 편차의 강력한 웨이블릿 기반 추정기입니다.
# denoise_tv_bregman : split-Bregman 최적화를 사용하여 총 변동 잡음 제거를 수행합니다.
# denoise_nl_means : 로컬이 아닌 수행은 2D-4D 그레이스케일 또는 RGB 이미지에서 노이즈 제거를 의미합니다.
# 출처 : https://scikit-image.org/docs/stable/api/skimage.restoration.html
from skimage.filters import gaussian
from skimage.color import rgb2gray

# Data reading and visualization
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler

# Training part
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, GlobalAveragePooling2D, Lambda
from keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator # 이미지 증강, ImageDataGenerator 클래스를 사용하면 rotation_range 인수에 정수 값을 제공하여 0~360도 범위에서 이미지를 임의로 회전할 수 있습니다.


# Any results you write to the current directory are saved as output.

In [26]:
# Translate data to an image format
def color_composite(data):
    rgb_arrays = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2
        
        # 백분위수
        r = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        g = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        b = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))

        rgb = np.dstack((r, g, b)) # https://yeko90.tistory.com/63
        rgb_arrays.append(rgb)
        
    return np.array(rgb_arrays)

def denoise(X, weight, multichannel):
    return np.asarray([denoise_tv_chambolle(item, weight=weight, multichannel=multichannel) for item in X])

def smooth(X, sigma):
    return np.asarray([gaussian(item, sigma=sigma) for item in X])

def grayscale(X):
    return np.asarray([rgb2gray(item) for item in X])

In [27]:
train = pd.read_json("./input/train.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
train_all = True

# 이것들은 모델을 더 효율적으로 훈련시키기 위해 필요한 train flag입니다.
# 적절한 모형 모수를 선택합니다.
train_b = True or train_all
train_img = True or train_all
train_total = True or train_all
predict_submission = True and train_all

clean_all = False
clean_b = False or clean_all
clean_img = False or clean_all

load_all = False
load_b = False or load_all
load_img = False or load_all

In [31]:
train.shape

(1604, 5)

In [29]:
color_composite(train).shape

(1604, 75, 75, 3)

In [39]:
band_1, band_2, images = train['band_1'].values, train['band_2'].values, color_composite(train)
to_arr = lambda x: np.asarray([np.asarray(item) for item in x]) # x=band_1,2
band_1 = to_arr(band_1) # (1604, 5625)
band_2 = to_arr(band_2) # (1604, 5625)
band_3 = (band_1 + band_2) / 2 # (1604, 5625)
gray_reshape = lambda x: np.asarray([item.reshape(75, 75) for item in x])

# Make a picture format from flat vector
band_1 = gray_reshape(band_1) # (1604, 75, 75)
band_2 = gray_reshape(band_2) # (1604, 75, 75)

tf_reshape = lambda x: np.asarray([item.reshape(75, 75, 1) for item in x])
band_1 = tf_reshape(band_1) # (1604, 75, 75, 1)
band_2 = tf_reshape(band_2) # (1604, 75, 75, 1)
band_3 = tf_reshape(band_3) # (1604, 75, 75, 1)

band = np.concatenate([band_1, band_2, band_3], axis=3)
band.shape # (1604, 75, 75, 3)

(1604, 75, 75, 3)

In [36]:
band_1, band_2, images = train['band_1'].values, train['band_2'].values, color_composite(train)
to_arr = lambda x: np.asarray([np.asarray(item) for item in x]) # x=band_1,2
band_1 = to_arr(band_1)
band_2 = to_arr(band_2)
band_3 = (band_1 + band_2) / 2
band_1.shape

(1604, 5625)

In [38]:
band_1.shape

(1604, 75, 75, 1)

In [ ]:
def create_dataset(frame, labeled, smooth_rgb=0.2, smooth_gray=0.5,
                   weight_rgb=0.05, weight_gray=0.05):
    
    band_1, band_2, images = train['band_1'].values, train['band_2'].values, color_composite(train)
    to_arr = lambda x: np.asarray([np.asarray(item) for item in x]) # x=band_1,2
    band_1 = to_arr(band_1) # (1604, 5625)
    band_2 = to_arr(band_2) # (1604, 5625)
    band_3 = (band_1 + band_2) / 2 # (1604, 5625)
    gray_reshape = lambda x: np.asarray([item.reshape(75, 75) for item in x])

    # Make a picture format from flat vector
    band_1 = gray_reshape(band_1) # (1604, 75, 75)
    band_2 = gray_reshape(band_2) # (1604, 75, 75)
    band_3 = gray_reshape(band_3) # (1604, 75, 75)
    print('Denoising and reshaping')
    
    if train_b and clean_b:
        # Smooth and denoise data
        band_1 = smooth(denoise(band_1, weight_gray, False), smooth_gray)
        print('Gray 1 done')
        band_2 = smooth(denoise(band_2, weight_gray, False), smooth_gray)
        print('Gray 2 done')
        band_3 = smooth(denoise(band_3, weight_gray, False), smooth_gray)
        print('Gray 3 done')
    
    if train_img and clean_img:
        images = smooth(denoise(images, weight_rgb, True), smooth_rgb)
    
    print('RGB done')
    
    tf_reshape = lambda x: np.asarray([item.reshape(75, 75, 1) for item in x])
    band_1 = tf_reshape(band_1) # (1604, 75, 75, 1)
    band_2 = tf_reshape(band_2) # (1604, 75, 75, 1)
    band_3 = tf_reshape(band_3) # (1604, 75, 75, 1)
    
    #images = tf_reshape(images)
    band = np.concatenate([band_1, band_2, band_3], axis=3) # (1604, 75, 75, 3)
    
    if labeled:
        y = np.array(frame["is_iceberg"])
    else:
        y = None
    return y, band, images

In [ ]:
y_train, X_b, X_images = create_dataset(train, True)